In [ ]:
! git clone https://github.com/uilab-kaist/cs475-mlnlp-spring-2021-hw.git

fatal: destination path 'cs475-mlnlp-spring-2021-hw' already exists and is not an empty directory.


# CS475 Spring 2021 Homework 3

This notebook is based on the [Huggingface's official example](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb). Note that you should submit `bert_poolers.py`, not `.ipynb` file converted from this colab notebook. Please refer to the homework description for more details.

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets transformers

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/text-classification).

**You have to implement the following block, which is the same as `bert_poolers.py`.**

In [ ]:
import torch
import torch.nn as nn
from transformers import BertConfig
from transformers.models.bert.modeling_bert import (
    BertPreTrainedModel, BertModel,
    BertEmbeddings, BertEncoder, BertForSequenceClassification, BertPooler,
)


class MeanMaxTokensBertPooler(nn.Module):
    # def __init__(self, config):
    #     super().__init__()
    #     # raise NotImplementedError


    # def forward(self, hidden_states, *args, **kwargs):
    #     raise NotImplementedError
    def __init__(self, config):
        super().__init__()
        self.mydense = nn.Linear(2*config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states, *args, **kwargs):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.

        max_token_tensor = torch.max(hidden_states,1)
        mean_token_tensor = torch.mean(hidden_states,1)
        #print(config.hidden_size)
        #print('hidden_states',hidden_states.shape)
        #print('max_token_tensor',max_token_tensor[0].shape)
        #print('mean_token_tensor',mean_token_tensor.shape)
        
        
        concat_token_tensor = torch.cat((max_token_tensor[0],mean_token_tensor), axis=1)
        #print('concat_token_tensor',concat_token_tensor.shape)

        pooled_output = self.mydense(concat_token_tensor)
        #print('total_tensor_stack1',torch.stack((max_token_tensor.values,mean_token_tensor),dim=1).shape)
        #print('total_tensor_stack0',torch.stack((max_token_tensor.values,mean_token_tensor),dim=0).shape)
        #print('total_tensor_cat1',torch.cat((max_token_tensor.values,mean_token_tensor),dim=1).shape)
        #print('total_tensor_cat0',torch.cat((max_token_tensor.values,mean_token_tensor),dim=0).shape)

        pooled_output = self.activation(pooled_output)
        return pooled_output

# [CLS] t1 t2 t3 [SEP] t4 t5 t6 t7 [SEP] 

# [CLS] || Max(Mean(Sent1), Mean(Sent2)...)

class MyBertPooler(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.mydense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states, *args, **kwargs):
        max_token_tensor = torch.topk(hidden_states,dim=1,k=config.k)
        
        pooled_output = self.mydense(torch.sum(max_token_tensor[0], dim=1)/config.k)

        pooled_output = self.activation(pooled_output)
        return pooled_output


class MyBertConfig(BertConfig):
    def __init__(self, pooling_layer_type="CLS", k=3, **kwargs):
        super().__init__(**kwargs)
        self.pooling_layer_type = pooling_layer_type
        self.k = k


class MyBertModel(BertModel):

    def __init__(self, config: MyBertConfig):
        super(BertModel, self).__init__(config)
        self.config = config

        self.embeddings = BertEmbeddings(config)
        self.encoder = BertEncoder(config)

        if config.pooling_layer_type == "CLS":
            # See src/transformers/models/bert/modeling_bert.py#L610
            # at huggingface/transformers (9f43a425fe89cfc0e9b9aa7abd7dd44bcaccd79a)
            self.pooler = BertPooler(config)
        elif config.pooling_layer_type == "MEAN_MAX":
            self.pooler = MeanMaxTokensBertPooler(config)
        elif config.pooling_layer_type == "MINE":
            self.pooler = MyBertPooler(config)
        else:
            raise ValueError(f"Wrong pooling_layer_type: {config.pooling_layer_type}")

        self.init_weights()

    @property
    def pooling_layer_type(self):
        return self.config.pooling_layer_type


class MyBertForSequenceClassification(BertForSequenceClassification):
    def __init__(self, config):
        super(BertForSequenceClassification, self).__init__(config)
        self.num_labels = config.num_labels

        self.bert = MyBertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):
        if self.bert.pooling_layer_type in ["CLS", "MEAN_MAX"]:
            return super().forward(
                input_ids, attention_mask, token_type_ids, position_ids, head_mask,
                inputs_embeds, labels, output_attentions, output_hidden_states, return_dict
            )
        else:
            # raise NotImplementedError
            return super().forward(
                input_ids, attention_mask, token_type_ids, position_ids, head_mask,
                inputs_embeds, labels, output_attentions, output_hidden_states, return_dict
            )

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).

![Widget inference on a text classification task](https://github.com/huggingface/notebooks/blob/master/examples/images/text_classification.png?raw=1)

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a  dataset containing sentences labeled grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [ ]:
task = "mrpc"
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [ ]:
from datasets import load_dataset, load_metric

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [ ]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

To access an actual element, you need to select a split first, then give an index:

In [ ]:
dataset["train"][0]

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    # display(HTML(df.to_html()))
    print(df)

In [ ]:
show_random_elements(dataset["train"])

    idx  ...                                          sentence2
0  2916  ...  Proceedings were taken up with prosecutors out...
1   509  ...  Officials in Chechnya 's Moscow-backed governm...
2   116  ...  The Standard & Poor 's 500 Index was up 1.75 p...
3  3371  ...  Maddox , who had battled cancer since 1983 , c...
4  1258  ...  PeopleSoft has argued that an Oracle takeover ...
5  1125  ...  Also Thursday , the NYSE 's board elected six ...
6  1020  ...  " We make no apologies for finding every legal...
7   640  ...  Spain , Poland , United Arab Emirates and Lith...
8  3347  ...  Palm Beach County is prepared to kick in anoth...
9   465  ...  Freeman said not having a driver license has b...

[10 rows x 4 columns]


The metric is an instance of [`datasets.Metric`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Metric):

In [ ]:
metric

Metric(name: "glue", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print(res

You can call its `compute` method with your predictions and labels directly and it will return a dictionary with the metric(s) value:

In [ ]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5625, 'f1': 0.6216216216216216}

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can double check it does work on our current dataset:

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence 1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
Sentence 2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .


We can them write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], [101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102, 9805, 3540, 11514, 2050, 4149, 11282, 2243, 1005, 1055, 1999, 2786, 2005, 1002, 6353, 2509, 2454, 1998, 2853, 2009, 2000, 3647, 4576, 2005, 1002, 1015, 1012, 1022, 4551, 1999, 2687, 1012, 102], [101, 2027, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 2006, 2238, 2184, 1010, 5378, 1996, 6636, 2005, 5096, 1010, 2002, 2794, 1012, 102, 2006, 2238, 2184, 1010, 1996, 2911, 1005, 1055, 5608, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 1010, 5378, 1996, 14792, 2005, 5096, 1012, 102], [101, 2105, 6021, 19481, 13938, 2102, 1010,

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the `MyBertForSequenceClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

**Here, you can change the most important argument in this assignment, `pooling_layer_type`.**

# Start here!

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

pooling_layer_type = "MINE"  # CLS, MEAN_MAX, MINE

config = MyBertConfig.from_pretrained(
    pretrained_model_name_or_path=model_checkpoint,
    pooling_layer_type=pooling_layer_type,
    k = 1,
)
model = MyBertForSequenceClassification.from_pretrained(model_checkpoint, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing MyBertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MyBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MyBertForSequenceCl

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the `Trainer` to load the best model it saved (according to `metric_name`) at the end of training.

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B):

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

You might wonder why we pass along the `tokenizer` when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The `tokenizer` has a pad method that will do all of this right for us, and the `Trainer` will use it. You can customize this part by defining and passing your own `data_collator` which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Runtime,Samples Per Second
1,No log,0.467804,0.779412,0.853896,3.073300,132.754000
2,No log,0.370523,0.833333,0.883162,3.075000,132.681000
3,0.466100,0.371406,0.852941,0.895833,3.020400,135.080000


TrainOutput(global_step=690, training_loss=0.40669128307397817, metrics={'train_runtime': 572.5973, 'train_samples_per_second': 1.205, 'total_flos': 548674012030992.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 441398784, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -2600960, 'train_mem_gpu_alloc_delta': 1769472512, 'train_mem_cpu_peaked_delta': 2981888, 'train_mem_gpu_peaked_delta': 949862400})

We can check with the `evaluate` method that our `Trainer` did reload the best model properly (if it was not the last one):

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.8529411764705882,
 'eval_f1': 0.8958333333333334,
 'eval_loss': 0.3714061677455902,
 'eval_mem_cpu_alloc_delta': 245760,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 66889216,
 'eval_runtime': 3.0714,
 'eval_samples_per_second': 132.837}

To see how your model fared you can compare it to the [GLUE Benchmark leaderboard](https://gluebenchmark.com/leaderboard).

In [ ]:
# CSL for mrpc
# {'epoch': 3.0,
#  'eval_accuracy': 0.8602941176470589,
#  'eval_f1': 0.9008695652173914,
#  'eval_loss': 0.40170010924339294,
#  'eval_mem_cpu_alloc_delta': 110592,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 67874304,
#  'eval_runtime': 2.9454,
#  'eval_samples_per_second': 138.523}

# MEANMAX 
# {'epoch': 3.0,
#  'eval_accuracy': 0.8504901960784313,
#  'eval_f1': 0.8957264957264958,
#  'eval_loss': 0.3684898912906647,
#  'eval_mem_cpu_alloc_delta': 24576,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 76175360,
#  'eval_runtime': 3.9338,
#  'eval_samples_per_second': 103.716}

# k=1
# {'epoch': 3.0,
#  'eval_accuracy': 0.8529411764705882,
#  'eval_f1': 0.8958333333333334,
#  'eval_loss': 0.3714061677455902,
#  'eval_mem_cpu_alloc_delta': 245760,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 66889216,
#  'eval_runtime': 3.0714,
#  'eval_samples_per_second': 132.837}

# k = 2, mrpc, {'epoch': 3.0,
#  'eval_accuracy': 0.8504901960784313,
#  'eval_f1': 0.8939130434782607,
#  'eval_loss': 0.34628328680992126,
#  'eval_mem_cpu_alloc_delta': 0,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 59504640,
#  'eval_runtime': 4.0963,
#  'eval_samples_per_second': 99.602}

# k=3
# {'epoch': 3.0,
#  'eval_accuracy': 0.8529411764705882,
#  'eval_f1': 0.8954703832752613,
#  'eval_loss': 0.40177130699157715,
#  'eval_mem_cpu_alloc_delta': 155648,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 66889216,
#  'eval_runtime': 3.1829,
#  'eval_samples_per_second': 128.186}

# k=4
# {'epoch': 3.0,
#  'eval_accuracy': 0.8529411764705882,
#  'eval_f1': 0.8961937716262977,
#  'eval_loss': 0.34824737906455994,
#  'eval_mem_cpu_alloc_delta': 1773568,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 49876480,
#  'eval_runtime': 3.0698,
#  'eval_samples_per_second': 132.909}

# k = 5 for mrpc 
#  {'epoch': 3.0,
#  'eval_accuracy': 0.8578431372549019,
#  'eval_f1': 0.901023890784983,
#  'eval_loss': 0.3723962903022766,
#  'eval_mem_cpu_alloc_delta': 114688,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 62537216,
#  'eval_runtime': 3.9636,
#  'eval_samples_per_second': 102.936}

# k = 6 for mrpc {'epoch': 3.0,
#  'eval_accuracy': 0.8627450980392157,
#  'eval_f1': 0.9037800687285222,
#  'eval_loss': 0.3457808494567871,
#  'eval_mem_cpu_alloc_delta': 57344,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 75643904,
#  'eval_runtime': 3.9755,
#  'eval_samples_per_second': 102.628}


# k = 7 {'epoch': 3.0,
#  'eval_accuracy': 0.8602941176470589,
#  'eval_f1': 0.9038785834738617,
#  'eval_loss': 0.4484735429286957,
#  'eval_mem_cpu_alloc_delta': 188416,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 47478272,
#  'eval_runtime': 4.0375,
#  'eval_samples_per_second': 101.053}

# k = 8 {'epoch': 3.0,
#  'eval_accuracy': 0.8602941176470589,
#  'eval_f1': 0.9042016806722689,
#  'eval_loss': 0.45298153162002563,
#  'eval_mem_cpu_alloc_delta': 1335296,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 48051200,
#  'eval_runtime': 3.9906,
#  'eval_samples_per_second': 102.242}

## Cola Task
# k = 2
# {'epoch': 3.0,
#  'eval_loss': 0.6560384035110474,
#  'eval_matthews_correlation': 0.6033168402681877,
#  'eval_mem_cpu_alloc_delta': 225280,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 31879168,
#  'eval_runtime': 5.8966,
#  'eval_samples_per_second': 176.882}

#MINE k=3
# {'epoch': 3.0,
#  'eval_loss': 0.6819043755531311,
#  'eval_matthews_correlation': 0.5711178585424906,
#  'eval_mem_cpu_alloc_delta': 122880,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 29571584,
#  'eval_runtime': 5.5627,
#  'eval_samples_per_second': 187.499}

# k = 4
# {'epoch': 3.0,
#  'eval_loss': 0.6627866625785828,
#  'eval_matthews_correlation': 0.5807606001872874,
#  'eval_mem_cpu_alloc_delta': 118784,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 19694080,
#  'eval_runtime': 7.4638,
#  'eval_samples_per_second': 139.741}

# k = 7
# {'epoch': 3.0,
#  'eval_loss': 0.6596372127532959,
#  'eval_matthews_correlation': 0.5885471185335819,
#  'eval_mem_cpu_alloc_delta': 81920,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 28574720,
#  'eval_runtime': 7.264,
#  'eval_samples_per_second': 143.584}

# MEANMAX
# {'epoch': 3.0,
#  'eval_loss': 0.4695543050765991,
#  'eval_matthews_correlation': 0.5728724145437311,
#  'eval_mem_cpu_alloc_delta': 61440,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 19973120,
#  'eval_runtime': 5.5181,
#  'eval_samples_per_second': 189.013}


# CLS
# {'epoch': 3.0,
#  'eval_loss': 0.46784117817878723,
#  'eval_matthews_correlation': 0.5760780218044883,
#  'eval_mem_cpu_alloc_delta': 487424,
#  'eval_mem_cpu_peaked_delta': 0,
#  'eval_mem_gpu_alloc_delta': 0,
#  'eval_mem_gpu_peaked_delta': 27947008,
#  'eval_runtime': 7.4359,
#  'eval_samples_per_second': 140.265}